# Navier's Solution to Simply-Supported Mindlin-Reissner Plates


The notebook goes through the steps of symbolicly getting the expressions that are turned to code in the library. The steps lack explanation at the moment, later I'll come back to it (the author).

In [1]:
from sigmaepsilon.math.linalg import inv_sym_3x3
import sympy as sy
from sympy import (
    symbols,
    Function,
    diff,
    Matrix,
    MatMul,
    integrate,
    Symbol,
    sin,
    cos,
    pi,
    simplify,
)

## Displacement Field


In [2]:
# coordinates in a suitable vector space
x, y, z = symbols("x y z", real=True)

# kinematic variables
w0 = Function("w_0")(x, y)
thx = Function("\\theta_x")(x, y)
thy = Function("\\theta_y")(x, y)

u = z * thy
v = -z * thx
w = w0
disps = [u, v, w]
Matrix(disps)

Matrix([
[ z*\theta_y(x, y)],
[-z*\theta_x(x, y)],
[        w_0(x, y)]])

## Boundary Conditions and Trial Solution


In [3]:
mn = m, n = symbols("m n", integer=True, positive=True)
coeffs = Amn, Bmn, Cmn = symbols("A_{mn} B_{mn} C_{mn}", real=True)
shape = Lx, Ly = symbols("L_x, L_y", real=True)
Sm, Sn = sin(m * pi * x / Lx), sin(n * pi * y / Ly)
Cm, Cn = cos(m * pi * x / Lx), cos(n * pi * y / Ly)

In [4]:
w0_trial = Amn * Sm * Sn
thx_trial = Bmn * Cn * Sm
thy_trial = Cmn * Sn * Cm
trial = {w0: w0_trial, thx: thx_trial, thy: thy_trial}
disps_trial = [s.subs(trial).expand().doit() for s in [w0_trial, thx_trial, thy_trial]]
Matrix(disps_trial)

Matrix([
[A_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[B_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[C_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)]])

## Strain - Displacement Equations


In [5]:
exx = diff(u, x)
eyy = diff(v, y)
exy = diff(u, y) + diff(v, x)
exz = diff(w, x) + diff(u, z)
eyz = diff(w, y) + diff(v, z)
strains = [exx, eyy, exy, exz, eyz]
Matrix(strains)

Matrix([
[                                   z*Derivative(\theta_y(x, y), x)],
[                                  -z*Derivative(\theta_x(x, y), y)],
[-z*Derivative(\theta_x(x, y), x) + z*Derivative(\theta_y(x, y), y)],
[                         \theta_y(x, y) + Derivative(w_0(x, y), x)],
[                        -\theta_x(x, y) + Derivative(w_0(x, y), y)]])

In [6]:
kxx = diff(exx, z)
kyy = diff(eyy, z)
kxy = diff(exy, z)
gen_strains = [kxx, kyy, kxy, exz, eyz]
Matrix(gen_strains)

Matrix([
[                                 Derivative(\theta_y(x, y), x)],
[                                -Derivative(\theta_x(x, y), y)],
[-Derivative(\theta_x(x, y), x) + Derivative(\theta_y(x, y), y)],
[                     \theta_y(x, y) + Derivative(w_0(x, y), x)],
[                    -\theta_x(x, y) + Derivative(w_0(x, y), y)]])

In [7]:
strains_trial = [s.subs(trial).expand().doit() for s in strains]
Matrix(strains_trial)

Matrix([
[                                                    -pi*C_{mn}*m*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                     pi*B_{mn}*n*z*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*B_{mn}*m*z*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*C_{mn}*n*z*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[              pi*A_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x + C_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[              pi*A_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y - B_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)]])

In [8]:
gen_strains_trial = [k.subs(trial).expand().doit() for k in gen_strains]
Matrix(gen_strains_trial)

Matrix([
[                                                  -pi*C_{mn}*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                   pi*B_{mn}*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*B_{mn}*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*C_{mn}*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          pi*A_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x + C_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[          pi*A_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y - B_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)]])

## Stress Resultants


In [9]:
C11, C22, C12, C16, C26, C66 = C126ij = symbols(
    "C_11 C_22 C_12 C_16 C_26 C_66", real=True
)
C126 = sy.Matrix([[C11, C12, 0], [C12, C22, 0], [0, 0, C66]])
sxx, syy, sxy = MatMul(C126, sy.Matrix([exx, eyy, exy])).doit()
C44, C55 = C45ij = symbols("C_44 C_55", real=True)
C45 = sy.Matrix([[C44, 0], [0, C55]])
syz, sxz = MatMul(C45, sy.Matrix([eyz, exz])).doit()

# integrate through the thickness
h = Symbol("t", real=True)  # thickness


def Int(expr):
    return integrate(expr, (z, -h / 2, h / 2))


D11, D22, D12, D66 = Dij = symbols("D_11 D_22 D_12 D_66", real=True)
S44, S55 = Sij = symbols("S_44 S_55", real=True)
#
mx, my, mxy = M = Matrix([Int(s * z) for s in [sxx, syy, sxy]])
vx, vy = V = Matrix([Int(s) for s in [sxz, syz]])
#
mx = mx.simplify().expand()
cD11 = mx.coeff(C11 * h**3 / 12)
cD12 = mx.coeff(C12 * h**3 / 12)
mx = D11 * cD11 + D12 * cD12
#
my = my.simplify().expand()
cD22 = my.coeff(C22 * h**3 / 12)
cD21 = my.coeff(C12 * h**3 / 12)
my = D22 * cD22 + D12 * cD21
#
mxy = mxy.simplify().expand()
cD66 = mxy.coeff(C66 * h**3 / 12)
mxy = D66 * cD66
#
vx = vx.simplify().expand()
cS55 = vx.coeff(C55 * h)
vx = S55 * cS55
#
vy = vy.simplify().expand()
cS44 = vy.coeff(C44 * h)
vy = S44 * cS44
mx = mx.simplify().expand()
my = my.simplify().expand()
mxy = mxy.simplify().expand()
vx = vx.simplify().expand()
vy = vy.simplify().expand()

In [10]:
gen_forces = [mx, my, mxy, vx, vy]
Matrix(gen_forces)

Matrix([
[ D_11*Derivative(\theta_y(x, y), x) - D_12*Derivative(\theta_x(x, y), y)],
[ D_12*Derivative(\theta_y(x, y), x) - D_22*Derivative(\theta_x(x, y), y)],
[-D_66*Derivative(\theta_x(x, y), x) + D_66*Derivative(\theta_y(x, y), y)],
[                     S_55*\theta_y(x, y) + S_55*Derivative(w_0(x, y), x)],
[                    -S_44*\theta_x(x, y) + S_44*Derivative(w_0(x, y), y)]])

In [11]:
gen_forces_trial = [k.subs(trial).expand().doit() for k in gen_forces]
Matrix(gen_forces_trial)

Matrix([
[ pi*B_{mn}*D_12*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*C_{mn}*D_11*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[ pi*B_{mn}*D_22*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*C_{mn}*D_12*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*B_{mn}*D_66*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*C_{mn}*D_66*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          pi*A_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x + C_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[          pi*A_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y - B_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)]])

In [12]:
D = sy.zeros(5, 5)
D[0, 0] = mx.simplify().coeff(kxx)
D[0, 1] = mx.simplify().coeff(kyy)
D[0, 2] = mx.simplify().coeff(kxy)
D[1, 0] = my.simplify().coeff(kxx)
D[1, 1] = my.simplify().coeff(kyy)
D[1, 2] = my.simplify().coeff(kxy)
D[2, 0] = mxy.simplify().coeff(kxx)
D[2, 1] = mxy.simplify().coeff(kyy)
D[2, 2] = mxy.simplify().coeff(kxy)
D[3, 3] = vx.simplify().coeff(exz)
D[3, 4] = vx.simplify().coeff(eyz)
D[4, 3] = vy.simplify().coeff(exz)
D[4, 4] = vy.simplify().coeff(eyz)
D

Matrix([
[D_11, D_12,    0,    0,    0],
[D_12, D_22,    0,    0,    0],
[   0,    0, D_66,    0,    0],
[   0,    0,    0, S_55,    0],
[   0,    0,    0,    0, S_44]])

## Equilibrium Equations


The signs of the equations
is selected such, that the coefficients of the load functions on the
right-hand sides admit positive signs according to global axes.


In [13]:
# vertical equilibrium
lhs_fz = simplify(-diff(vx, x) - diff(vy, y)).expand()
# moment around global X
lhs_mx = simplify(diff(mxy, x) + diff(my, y) - vy).expand()
# moment around global Y
lhs_my = simplify(-diff(mxy, y) - diff(mx, x) + vx).expand()

equations = [lhs_fz, lhs_mx, lhs_my]
Matrix(equations)

Matrix([
[                                                                       S_44*Derivative(\theta_x(x, y), y) - S_44*Derivative(w_0(x, y), (y, 2)) - S_55*Derivative(\theta_y(x, y), x) - S_55*Derivative(w_0(x, y), (x, 2))],
[ D_12*Derivative(\theta_y(x, y), x, y) - D_22*Derivative(\theta_x(x, y), (y, 2)) - D_66*Derivative(\theta_x(x, y), (x, 2)) + D_66*Derivative(\theta_y(x, y), x, y) + S_44*\theta_x(x, y) - S_44*Derivative(w_0(x, y), y)],
[-D_11*Derivative(\theta_y(x, y), (x, 2)) + D_12*Derivative(\theta_x(x, y), x, y) - D_66*Derivative(\theta_y(x, y), (y, 2)) + D_66*Derivative(\theta_x(x, y), x, y) + S_55*\theta_y(x, y) + S_55*Derivative(w_0(x, y), x)]])

In [14]:
equations_trial = [k.subs(trial).expand().doit() for k in equations]
Matrix(equations_trial)

Matrix([
[                                                                                                                            pi**2*A_{mn}*S_44*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2 + pi**2*A_{mn}*S_55*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 - pi*B_{mn}*S_44*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y + pi*C_{mn}*S_55*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*A_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y + pi**2*B_{mn}*D_22*n**2*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y**2 + pi**2*B_{mn}*D_66*m**2*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x**2 + B_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y) - pi**2*C_{mn}*D_12*m*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y) - pi**2*C_{mn}*D_66*m*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[ pi*A_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x - pi**2*B_{mn}*D_12*m*n*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y) - pi**2*B_{mn}*D_66*m*n*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y) + pi**2*C_{mn}*D_11*m**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x**2 + pi**2*C_{mn}

In [23]:
eq_fz_trial = equations_trial[0].expand().doit() / (Sm * Sn)
eq_fz_trial = eq_fz_trial.simplify().expand()
eq_mx_trial = equations_trial[1].expand().doit() / (Sm * Cn)
eq_mx_trial = eq_mx_trial.simplify().expand()
eq_my_trial = equations_trial[2].expand().doit() / (Cm * Sn)
eq_my_trial = eq_my_trial.simplify().expand()

P = sy.zeros(3, 3)
P[0, :] = Matrix([eq_fz_trial.coeff(c) for c in coeffs]).T
P[1, :] = Matrix([eq_mx_trial.coeff(c) for c in coeffs]).T
P[2, :] = Matrix([eq_my_trial.coeff(c) for c in coeffs]).T
detP, adjP = inv_sym_3x3(P, as_adj_det=True)
detP = detP.simplify().expand()
adjP.simplify()
P

Matrix([
[pi**2*S_44*n**2/L_y**2 + pi**2*S_55*m**2/L_x**2,                                         -pi*S_44*n/L_y,                                          pi*S_55*m/L_x],
[                                 -pi*S_44*n/L_y, pi**2*D_22*n**2/L_y**2 + pi**2*D_66*m**2/L_x**2 + S_44,   -pi**2*D_12*m*n/(L_x*L_y) - pi**2*D_66*m*n/(L_x*L_y)],
[                                  pi*S_55*m/L_x,   -pi**2*D_12*m*n/(L_x*L_y) - pi**2*D_66*m*n/(L_x*L_y), pi**2*D_11*m**2/L_x**2 + pi**2*D_66*n**2/L_y**2 + S_55]])

### Postprocessing


In [24]:
Matrix([w0_trial, thx_trial, thy_trial])

Matrix([
[A_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[B_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[C_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)]])

In [25]:
kxx = kxx.subs(trial).expand().doit()
kxx = kxx.simplify().expand()
kyy = kyy.subs(trial).expand().doit()
kyy = kyy.simplify().expand()
kxy = kxy.subs(trial).expand().doit()
kxy = kxy.simplify().expand()
exz = exz.subs(trial).expand().doit()
exz = exz.simplify().expand()
eyz = eyz.subs(trial).expand().doit()
eyz = eyz.simplify().expand()
Matrix([kxx, kyy, kxy, exz, eyz])

Matrix([
[                                                  -pi*C_{mn}*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[                                                   pi*B_{mn}*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y],
[-pi*B_{mn}*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*C_{mn}*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          pi*A_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x + C_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[          pi*A_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y - B_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)]])

In [26]:
mx = mx.subs(trial).expand().doit()
mx = mx.simplify().expand()
my = my.subs(trial).expand().doit()
my = my.simplify().expand()
mxy = mxy.subs(trial).expand().doit()
mxy = mxy.simplify().expand()
vx = vx.subs(trial).expand().doit()
vx = vx.simplify().expand()
vy = vy.subs(trial).expand().doit()
vy = vy.simplify().expand()
Matrix([mx, my, mxy, vx, vy])

Matrix([
[ pi*B_{mn}*D_12*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*C_{mn}*D_11*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[ pi*B_{mn}*D_22*n*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y - pi*C_{mn}*D_12*m*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x],
[-pi*B_{mn}*D_66*m*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_x + pi*C_{mn}*D_66*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[          pi*A_{mn}*S_55*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x + C_{mn}*S_55*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)],
[          pi*A_{mn}*S_44*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y - B_{mn}*S_44*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)]])

### Loads


#### Constans vertical load over a rectangular area


In [27]:
# constans vertical load over a rectangular area
q, w, h = symbols("q w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    q * Sm * Sn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_fz = qmn.simplify().expand()
qmn_fz

16*q*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n)

#### Constans $m_x$ moment over a rectangular area


In [28]:
# constans moment of intensity mx, around global axis X,
# over a rectangular area with width w, height h and center (xc, yc)
m_x, w, h = symbols("m_x w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    m_x * Cm * Sn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_mx = qmn.simplify().expand()
qmn_mx

16*m_x*sin(pi*m*w/(2*L_x))*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)*cos(pi*m*x_c/L_x)/(pi**2*m*n)

#### Constans $m_y$ moment over a rectangular area


In [29]:
# constant moment of intensity my, around global axis Y,
# over a rectangular area with width w, height h and center (xc, yc)
m_y, w, h = symbols("m_y w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    m_y * Sm * Cn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_my = qmn.simplify().expand()
qmn_my

16*m_y*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*cos(pi*n*y_c/L_y)/(pi**2*m*n)

In [30]:
f = Matrix([qmn_fz, qmn_mx, qmn_my])
f

Matrix([
[  16*q*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n)],
[16*m_x*sin(pi*m*w/(2*L_x))*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)*cos(pi*m*x_c/L_x)/(pi**2*m*n)],
[16*m_y*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*cos(pi*n*y_c/L_y)/(pi**2*m*n)]])